In [1]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)

from main import *
from jax import jit
from functools import partial
import time as tm
# Bi modules
bi = bee(platform='cpu')
print(bee.dist.normal(0,1, sample = True, shape=(1,), seed = 1))
m.net.mat_to_edgl(jnp.array([[1, 2, 3, 4],
                              [5, 6, 7, 8],
                              [9, 10, 11, 12],
                              [13, 14, 15, 16]]))
print(BEE.gaussian_process)
print(BEE.random_centered)

C:\Users\SSosa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


jax.local_device_count 16
[-1.1842843]
<function Mgaussian.gaussian_process at 0x000001CAA6D56B60>
<PjitFunction of <function factors.random_centered at 0x000001CAA6D237E0>>


## Latent variable model

In [104]:
init_key, sample_key = random.split(random.PRNGKey(0))
init_key = jnp.array(init_key)
init_key

Array([4146024105,  967050713], dtype=uint32)

In [105]:
import pandas as pd 
import math
import os
import seaborn as sns
import arviz as az
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import Image, set_matplotlib_formats
from matplotlib.patches import Ellipse, transforms
import jax
import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import expit

import numpy as onp
import numpyro as numpyro

import numpyro.distributions as dist
from numpyro.diagnostics import effective_sample_size, print_summary
from numpyro.infer import MCMC, NUTS
numpyro.set_platform("cpu")
numpyro.set_host_device_count(30)


# Simulation ---------------
NY = 4
NV = 8

Y2 = np.full((NV, NY), np.nan) 
means = np.random.normal(0, 1, NY)
offsets = np.random.normal(0, 1, NV)
for i in range(NV):
  for k in range(NY):
    Y2[i,k] = means[k] + offsets[i]

b_individual = np.random.normal(0, 1, (N, K))
mu = b_individual + a

Y2 = jnp.array(Y2)



In [106]:

# Building model and sampling it ------------------
def model(NY, NV, Y2):
    means =  numpyro.sample('means', numpyro.distributions.Normal(0,1).expand([NY]))
    offset =  numpyro.sample('offset', numpyro.distributions.Normal(0,1).expand([NV,1]))
    sigma =  numpyro.sample('sigma', numpyro.distributions.Exponential(1).expand([NY])) 
    tmp = jnp.tile(means, (NV, 1)).reshape(NV,NY)  
    mu_l = tmp + offset 
    numpyro.sample('Y2', numpyro.distributions.Normal(mu_l, jnp.tile(sigma, [NV, 1])), obs=Y2)

dat = dict(
    NY = NY,
    NV = NV,
    Y2 = Y2
)
m = MCMC(NUTS(model, init_strategy = numpyro.infer.init_to_median()), num_warmup=1000, num_samples=1000, num_chains=1)
m.run(random.PRNGKey(0), extra_fields=["diverging"], **dat)

sample: 100%|██████████| 2000/2000 [00:01<00:00, 1386.84it/s, 1023 steps of size 1.50e-07. acc. prob=0.79]


In [31]:
import arviz as az
data = az.from_numpyro(m)
az.summary(data)

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
means[0],0.519,0.0,0.519,0.519,0.0,0.0,1000.0,1000.0,NaN
means[1],0.191,0.0,0.191,0.191,0.0,0.0,1000.0,1000.0,NaN
means[2],0.657,0.0,0.657,0.657,0.0,0.0,1000.0,1000.0,NaN
means[3],-0.613,0.0,-0.613,-0.613,0.0,0.0,1000.0,1000.0,NaN
"offset[0, 0]",0.530,0.0,0.530,0.530,0.0,0.0,1000.0,1000.0,NaN
"offset[1, 0]",0.812,0.0,0.812,0.812,0.0,0.0,1000.0,1000.0,NaN
"offset[2, 0]",-0.366,0.0,-0.366,-0.366,0.0,0.0,1000.0,1000.0,NaN
"offset[3, 0]",2.077,0.0,2.077,2.077,0.0,0.0,1000.0,1000.0,NaN
"offset[4, 0]",0.988,0.0,0.988,0.988,0.0,0.0,1000.0,1000.0,NaN
"offset[5, 0]",0.342,0.0,0.342,0.342,0.0,0.0,1000.0,1000.0,NaN


In [32]:
means

array([1.28912305, 0.96183663, 1.42779941, 0.15728591])